In [1]:
import time
import random
import requests
import pandas as pd
import json
from pandas import json_normalize

In [2]:
class Job104Spider():
    def search(self, keyword, max_nun=10, filter_params=None, sort_type='符合度', is_sort_asc=False):
        # 產生搜尋結果
        jobs = []
        total_count = 0

        url = 'https://www.104.com.tw/jobs/search/list'
        # 這個url不是從網址來的，要到F12>XHR>Headers中查找，所以一定要最後面的list
        query = f'ro=0&kwop=7&keyword={keyword}&expansionType=area,spec,com,job,wf,wktm&mode=s&jobsource=2018indexpoc'
        if filter_params:
            # 加上篩選參數，要先轉換為 URL 參數字串格式
            query += ''.join([f'&{key}={value}' for key, value, in filter_params.items()])
            
        # 加上排序條件
        sort_dict = {
            '符合度': '1',
            '日期': '2',
            '經歷': '3',
            '學歷': '4',
            '應徵人數': '7',
            '待遇': '13',
        }
        
        # 結果排序
        sort_params = f"&order={sort_dict.get(sort_type, '1')}"
        sort_params += '&asc=1' if is_sort_asc else '&asc=0'
        query += sort_params
            
        # 設定headers，偽裝成真人搜尋，避免被網頁偵測到
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
            'Referer': 'https://www.104.com.tw/jobs/search/',
        }
        
        page = 1
        while len(jobs) < max_nun:
            params = f'{query}&page={page}'
            r = requests.get(url, params=params, headers=headers)
            if r.status_code != requests.codes.ok:
                print('請求失敗', r.status_code)
                data = r.json()
                print(data['status'], data['statusMsg'], data['errorMsg'])
                break

            data = r.json()
            total_count = data['data']['totalCount']
            jobs.extend(data['data']['list'])

            if (page == data['data']['totalPage']) or (data['data']['totalPage'] == 0):
                break
            page += 1
            time.sleep(random.uniform(3, 5))

        return total_count, jobs[:max_nun]

In [3]:
job104_spider = Job104Spider()

In [4]:
filter_params = {
    'area': '6001001000,6001002000',  # (地區) 台北市,新北市
    # 's9': '1,2,4,8',  # (上班時段) 日班,夜班,大夜班,假日班
    # 's5': '0',  # 0:不需輪班 256:輪班
    # 'wktm': '1',  # (休假制度) 週休二日
    # 'isnew': '0',  # (更新日期) 0:本日最新 3:三日內 7:一週內 14:兩週內 30:一個月內
    # 'jobexp': '1,3,5,10,99',  # (經歷要求) 1年以下,1-3年,3-5年,5-10年,10年以上
    # 'newZone': '1,2,3,4,5',  # (科技園區) 竹科,中科,南科,內湖,南港
    # 'zone': '16',  # (公司類型) 16:上市上櫃 5:外商一般 4:外商資訊
    # 'wf': '1,2,3,4,5,6,7,8,9,10',  # (福利制度) 年終獎金,三節獎金,員工旅遊,分紅配股,設施福利,休假福利,津貼/補助,彈性上下班,健康檢查,團體保險
    # 'edu': '1,2,3,4,5,6',  # (學歷要求) 高中職以下,高中職,專科,大學,碩士,博士
    # 'remoteWork': '1',  # (上班型態) 1:完全遠端 2:部分遠端
    # 'excludeJobKeyword': '科技',  # 排除關鍵字
    # 'kwop': '1',  # 只搜尋職務名稱
    }

In [5]:
total_count, jobs= job104_spider.search('資料科學', max_nun=10, filter_params=filter_params)

In [6]:
print('搜尋結果職缺總數：', total_count)

搜尋結果職缺總數： 600


In [7]:
s1 = json.dumps(jobs)
d2 = json.loads(s1)
df = json_normalize(d2)

In [8]:
df

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major,link.applyAnalyze,link.job,link.cust
0,1,10503230,Android Engineer,Android Engineer,1,1,6001001004,台北市松山區,南京東路三段303巷3弄8號2樓,WeMo Scooter 2016年上路，以智慧出行串連智慧生活大小事，提供「自由租 輕鬆停...,...,hotjob_chr,Android Engineer,威摩科技股份有限公司,121.546017,25.052644,0,[],//www.104.com.tw/jobs/apply/analysis/694ce?cha...,//www.104.com.tw/job/694ce?jobsource=hotjob_chr,//www.104.com.tw/company/1a2x6bjhm7?jobsource=...
1,0,11774922,資料科學家,<em class='b-txt--highlight'>資料科學</em>家,1,1,6001001003,台北市中山區,中山北路二段61號11樓,[[[資料科學]]]家之職務內容如下：\n1. 協助集團發展銷售預測分析模型\n2. 協助集...,...,jolist_d_relevance,資料科學家,精藤股份有限公司,121.522992,25.0561769,0,"[應用數學相關, 統計學相關, 資訊管理相關]",//www.104.com.tw/jobs/apply/analysis/70dl6?cha...,//www.104.com.tw/job/70dl6?jobsource=jolist_d_...,//www.104.com.tw/company/18jr0l0o?jobsource=jo...
2,0,12001737,資料科學家,<em class='b-txt--highlight'>資料科學</em>家,1,1,6001001010,台北市內湖區,瑞光路335號6樓,期待有想法有衝勁的夥伴能夠加入我們。\n\n(1). 建構數據分析環境，並運用Python分...,...,jolist_d_relevance,資料科學家,台灣資料科學股份有限公司,121.5748181,25.0775884,0,"[數理統計相關, 資訊工程相關, 其他數學及電算機科學相關]",//www.104.com.tw/jobs/apply/analysis/758ll?cha...,//www.104.com.tw/job/758ll?jobsource=jolist_d_...,//www.104.com.tw/company/1a2x6bkvhu?jobsource=...
3,0,12474640,資料科學家,<em class='b-txt--highlight'>資料科學</em>家,1,1,6001001005,台北市大安區,敦化南路一段339號10樓,BI軟體進行[[[資料]]]視覺化儀表板設計。\n6. 研究客戶樣貌並結合數據技術，發展金...,...,jolist_d_relevance,資料科學家,裕融企業股份有限公司,121.5492679,25.0334879,0,"[資訊管理相關, 統計學相關, 應用數學相關]",//www.104.com.tw/jobs/apply/analysis/7fdhs?cha...,//www.104.com.tw/job/7fdhs?jobsource=jolist_d_...,//www.104.com.tw/company/at41hao?jobsource=jol...
4,0,11065262,資深資料科學家,資深<em class='b-txt--highlight'>資料科學</em>家,1,1,6001001003,台北市中山區,中山北路二段61號11樓,大家庭。\r\n\r\n本公司【資深[[[資料科學]]]家】之職務內容如下：\r\n1. 了...,...,jolist_d_relevance,資深資料科學家,精藤股份有限公司,121.522992,25.0561769,0,"[應用數學相關, 統計學相關, 資訊管理相關]",//www.104.com.tw/jobs/apply/analysis/6l60e?cha...,//www.104.com.tw/job/6l60e?jobsource=jolist_d_...,//www.104.com.tw/company/18jr0l0o?jobsource=jo...
5,0,12278726,資料科學家-Data Scientist,<em class='b-txt--highlight'>資料科學</em>家-Data S...,1,1,6001001003,台北市中山區,民生東路二段176號4樓,systems.\n\n任職要求：\n● [[[資料科學]]]、統計數學、資訊或量化分析相...,...,jolist_d_relevance,資料科學家-Data Scientist,大慶證券股份有限公司_總管理處,121.5358163,25.0576527,0,[],//www.104.com.tw/jobs/apply/analysis/7b6bq?cha...,//www.104.com.tw/job/7b6bq?jobsource=jolist_d_...,//www.104.com.tw/company/ajo2a6o?jobsource=jol...
6,0,11544847,資料科學家 Data Scientist,<em class='b-txt--highlight'>資料科學</em>家 Data S...,1,1,6001001003,台北市中山區,,這份職位將參與知名消費電子品牌數據分析部門專案，與多位[[[資料科學]]]家、[[[資料]]...,...,jolist_d_relevance,資料科學家 Data Scientist,干城數碼有限公司,121.5427093,25.0792018,0,"[資訊工程相關, 資訊管理相關, 數理統計相關]",//www.104.com.tw/jobs/apply/analysis/6vg27?cha...,//www.104.com.tw/job/6vg27?jobsource=jolist_d_...,//www.104.com.tw/company/1a2x6bk6fu?jobsource=...
7,0,12568192,資料科學家 / 數據分析師 暨 專案經理,<em class='b-txt--highlight'>資料科學</em>家 / 數據分析...,1,1,6001001003,台北市中山區,松江路101號10樓,✅ 公司簡介：\n「騰創數析」係由零售解決方案領導廠商「騰雲科技」，結盟[[[資料科學]]]...,...,jolist_d_relevance,資料科學家 / 數據分析師 暨 專案經理,騰創數析股份有限公司,121.5332717,25.0511511,0,[],//www.104.com.tw/jobs/apply/analysis/7hdog?cha...,//www.104.com.tw/job/7hdog?jobsource=jolist_d_...,//www.104.com.tw/company/1a2x6blvcu?jobsource=...
8,0,12302634,[VR BU] 資料科學家 Data Scientist,[VR BU] <em class='b-txt--highlight'>資料科學</em>...,1,1,6001001004,台北市松山區,南京東路4段2號2樓（小巨蛋南區入口處）,1.利用[[[資料科學]]]技術洞察客戶行為數據，創造商業價值輔以商務決策\n2.具有獨立工...,...,jolist_d_relevance,[VR BU] 資料科學家 Data Scientist,iStaging愛實境_宅妝股份有限公司,121.5504786,25.0510986,0,[],//www.104.com.tw/jobs/apply/analysis/7boru?cha...,//www.104.com.tw/job/7boru?jobsource=jolist_d_...,//www.104.com.tw/company/bjdang8?jobsource=jol...
9,0,11817751,AI資料科學家,AI<em class='b-txt--highlight'>資料科學</em>家,1,1,6001001011,台北市南港區,或龜山區,1.人工智慧演算法設計與開發 \n2.影像處理演算法設計與開發 \n 至少完成過一個...,...,jolist_d_relevance,AI資料科學家,展市華科技有限公司,121.6181159,25.0566019,0,[],//www.104.com.tw/jobs/apply/analysis/71a

In [9]:
df.columns

Index(['jobType', 'jobNo', 'jobName', 'jobNameSnippet', 'jobRole', 'jobRo',
       'jobAddrNo', 'jobAddrNoDesc', 'jobAddress', 'description', 'optionEdu',
       'period', 'periodDesc', 'applyCnt', 'applyDesc', 'custNo', 'custName',
       'coIndustry', 'coIndustryDesc', 'salaryLow', 'salaryHigh', 'salaryDesc',
       's10', 'appearDate', 'appearDateDesc', 'optionZone', 'isApply',
       'applyDate', 'isSave', 'descSnippet', 'tags', 'landmark', 'jobsource',
       'jobNameRaw', 'custNameRaw', 'lon', 'lat', 'remoteWorkType', 'major',
       'link.applyAnalyze', 'link.job', 'link.cust'],
      dtype='object')

In [10]:
df[['jobName', 'description', 'optionEdu',
       'periodDesc', 'applyDesc', 'custName',
       'coIndustryDesc', 'salaryDesc', 'tags', 'remoteWorkType', 'major']]

,jobName,description,optionEdu,periodDesc,applyDesc,custName,coIndustryDesc,salaryDesc,tags,remoteWorkType,major
0,Android Engineer,WeMo Scooter 2016年上路，以智慧出行串連智慧生活大小事，提供「自由租 輕鬆停...,大學,3年以上,6~10人應徵,威摩科技股份有限公司,,"年薪700,000~1,100,000元",[員工200人],0,[]
1,資料科學家,[[[資料科學]]]家之職務內容如下：\n1. 協助集團發展銷售預測分析模型\n2. 協助集...,大學,3年以上,6~10人應徵,精藤股份有限公司,電腦系統整合服務業,"年薪600,000~1,200,000元",[員工140人],0,"[應用數學相關, 統計學相關, 資訊管理相關]"
2,資料科學家,期待有想法有衝勁的夥伴能夠加入我們。\n\n(1). 建構數據分析環境，並運用Python分...,大學,經歷不拘,11~30人應徵,台灣資料科學股份有限公司,電腦系統整合服務業,"月薪38,000~88,000元",[員工190人],0,"[數理統計相關, 資訊工程相關, 其他數學及電算機科學相關]"
3,資料科學家,BI軟體進行[[[資料]]]視覺化儀表板設計。\n6. 研究客戶樣貌並結合數據技術，發展金...,大學,1年以上,6~10人應徵,裕融企業股份有限公司,其他汽機車相關業,"月薪45,000~60,000元","[上市上櫃, 員工600人]",0,"[資訊管理相關, 統計學相關, 應用數學相關]"
4,資深資料科學家,大家庭。\r\n\r\n本公司【資深[[[資料科學]]]家】之職務內容如下：\r\n1. 了...,碩士,5年以上,6~10人應徵,精藤股份有限公司,電腦系統整合服務業,"年薪1,200,000~1,800,000元",[員工140人],0,"[應用數學相關, 統計學相關, 資訊管理相關]"
5,資料科學家-Data Scientist,systems.\n\n任職要求：\n● [[[資料科學]]]、統計數學、資訊或量化分析相...,大學,經歷不拘,11~30人應徵,大慶證券股份有限公司_總管理處,證券及期貨業,待遇面議,"[上市上櫃, 員工350人]",0,[]
6,資料科學家 Data Scientist,這份職位將參與知名消費電子品牌數據分析部門專案，與多位[[[資料科學]]]家、[[[資料]]...,大學,2年以上,11~30人應徵,干城數碼有限公司,電腦軟體服務業,待遇面議,[],0,"[資訊工程相關, 資訊管理相關, 數理統計相關]"
7,資料科學家 / 數據分析師 暨 專案經理,✅ 公司簡介：\n「騰創數析」係由零售解決方案領導廠商「騰雲科技」，結盟[[[資料科學]]]...,大學,經歷不拘,11~30人應徵,騰創數析股份有限公司,工商顧問服務業,待遇面議,[員工18人],0,[]
8,[VR BU] 資料科學家 Data Scientist,1.利用[[[資料科學]]]技術洞察客戶行為數據，創造商業價值輔以商務決策\n2.具有獨立工...,大學,經歷不拘,0~5人應徵,iStaging愛實境_宅妝股份有限公司,其它軟體及網路相關業,待遇面議,[員工50人],0,[]
9,AI資料科學家,1.人工智慧演算法設計與開發 \n2.影像處理演算法設計與開發 \n 至少完成過一個...,大學,經歷不拘,6~10人應徵,展市華科技有限公司,電腦軟體服務業,"月薪35,000~70,000元",[],0,[]
